# TensorRT

NVIDIA® TensorRT™, an SDK for high-performance deep learning inference, includes a deep learning inference optimizer and runtime that delivers low latency and high throughput for inference applications.


This session shares required codes to how you can take an existing ONNX model built with a deep learning framework and build a TensorRT engine using the provided parsers. The Developer Guide also provides step-by-step instructions for common user tasks such as creating a TensorRT network definition, invoking the TensorRT builder, serializing and deserializing, and how to feed the engine with data and perform inference.

Since runnning these code requires NVIDIA GPU, we used Google Colab. Please keep in mind (based on my experience) generating TensorRT worked based on which NVIDIA GPU you use. It means you cannot generate a TensorRT Engine on one GPU and use it in another GPU. 

### Install requried libraries in Google Colab

In [ ]:
!pip install pycuda # install cuda
!pip install tensorrt

#### Check TensorRT version

In [ ]:
import tensorrt as trt
trt.__version__

#### Import requried libraries

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import os
import argparse
import tensorrt as trt
import random
import argparse
import os

import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
import tensorrt as trt
import numpy as np
from PIL import Image

#### Define Constants

In [ ]:
verbose = False
TRT_LOGGER = trt.Logger(trt.Logger.VERBOSE) if verbose else trt.Logger()
MAX_BATCH_SIZE = 1
input_name = 'input.1'
Input_shape = (3,608,608)
model_name = "model"
fp16 = False
int8 = False
dla_core = -1 
verbose = False
engine_path = '%s.trt' % model_name

#### Load created ONNX model 

In [ ]:
def load_onnx(model_name):
    """Read the ONNX file."""
    onnx_path = '%s.onnx' % model_name
    if not os.path.isfile(onnx_path):
        print('ERROR: file (%s) not found!  You might want to run yolo_to_onnx.py first to generate it.' % onnx_path)
        return None
    else:
        with open(onnx_path, 'rb') as f:
            return f.read()

In [ ]:
onnx_data = load_onnx(model_name)

#### Set network input batch size.

In [ ]:
def set_net_batch(network, batch_size):
    """Set network input batch size.

    The ONNX file might have been generated with a different batch size,
    say, 64.
    """
    if trt.__version__[0] >= '7':
        shape = list(network.get_input(0).shape)
        shape[0] = batch_size
        network.get_input(0).shape = shape
    return network

#### Build a TensorRT engine from ONNX.

In [ ]:
def build_engine(model_name,do_fp16, do_int8, dla_core, verbose=False):
    print('Loading the ONNX file...')
    onnx_data = load_onnx(model_name)
    if onnx_data is None:
        return None

    TRT_LOGGER = trt.Logger(trt.Logger.VERBOSE) if verbose else trt.Logger()
    EXPLICIT_BATCH = [] if trt.__version__[0] < '7' else \
        [1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)]
    with trt.Builder(TRT_LOGGER) as builder, builder.create_network(*EXPLICIT_BATCH) as network, trt.OnnxParser(network, TRT_LOGGER) as parser:
        if do_int8 and not builder.platform_has_fast_int8:
            raise RuntimeError('INT8 not supported on this platform')
        if not parser.parse(onnx_data):
            print('ERROR: Failed to parse the ONNX file.')
            for error in range(parser.num_errors):
                print(parser.get_error(error))
            return None
        network = set_net_batch(network, MAX_BATCH_SIZE)


        print('Building the TensorRT engine.  This would take a while...')
        print('(Use "--verbose" or "-v" to enable verbose logging.)')
        # new API: build_engine() with builder config
        builder.max_batch_size = MAX_BATCH_SIZE
        config = builder.create_builder_config()
        config.max_workspace_size = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
        config.set_flag(trt.BuilderFlag.GPU_FALLBACK)
        profile = builder.create_optimization_profile()
        profile.set_shape(input_name, (MAX_BATCH_SIZE, Input_shape[0], Input_shape[1], Input_shape[2]),
                          (MAX_BATCH_SIZE, Input_shape[0], Input_shape[1], Input_shape[2]),(MAX_BATCH_SIZE, Input_shape[0], Input_shape[1], Input_shape[2]))
        config.add_optimization_profile(profile)
        engine = builder.build_engine(network, config)
        if engine is not None:
            print('Completed creating engine.')
        return engine

#### Create the Engine

In [ ]:
engine = build_engine(model_name, fp16, int8, dla_core, verbose)
if engine is None:
      raise SystemExit('ERROR: failed to build the TensorRT engine!')
with open(engine_path, 'wb') as f:
      f.write(engine.serialize())
print('Serialized the TensorRT engine to file: %s' % engine_path)